## Part A ( i ) Baseline on Dataset I

## Part A ( ii ) Numpy Extension on Dataset I

## Compare Part A Results


## Part B ( i ) Baseline on Dataset II

## Part B ( ii ) Numpy Extension on Dataset II

## Compare Part B Results

## Summary of Results

In [0]:
print('test')